In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import seaborn as sn

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys,os
#sys.path.append('..')
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import datetime
import pandas as pd
import numpy as np
import scipy

In [3]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)

#INSTRUMENTS_LIST = ['ES', 'CL', 'NG', 'ZN', 'ZS', 'ZW', 'ZC', '6E', 'CC']
# INSTRUMENTS_LIST = ['CL']

#exo_filter = 'neutralOnly'     # All 
#exo_filter = 'ZC'  # ES only
exo_filter = 'ContFut'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

# Do quick sanity checks
for exo in sorted(storage.exo_list(exo_filter)):
    #print(exo)
    exo_df, exo_dict = storage.load_series(exo)
    
    if len(exo_df) < 200:
        print("{0} [NODATA DataLen: {1}]".format(exo, len(exo_df)))
        #storage.delete_exo(exo)
    elif (datetime.datetime.now() - exo_df.index[-1]).days > 4:
        print("{0} [DELAYED: LastDate: {1}]".format(exo, exo_df.index[-1]))
        #storage.delete_exo(exo)
    else:
        print("{0} [OK]".format(exo))

6A_ContFut [OK]
6B_ContFut [OK]
6C_ContFut [OK]
6E_ContFut [OK]
6J_ContFut [OK]
AL1_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
AW6_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
B6L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
CC_ContFut [OK]
CL_ContFut [OK]
DC_ContFut [OK]
E4L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
ES_ContFut [OK]
GC_ContFut [OK]
HE_ContFut [OK]
HG_ContFut [OK]
HO_ContFut [OK]
KC_ContFut [DELAYED: LastDate: 2016-07-11 00:00:00]
L3L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
LBS_ContFut [OK]
LE_ContFut [OK]
N9L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
NG_ContFut [OK]
NQ_ContFut [OK]
R7L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
RB_ContFut [OK]
SB_ContFut [OK]
SI_ContFut [OK]
V3L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
VX_ContFut [OK]
XAB_ContFut [OK]
XAE_ContFut [OK]
XAF_ContFut [OK]
XAI_ContFut [OK]
XAK_ContFut [OK]
XAP_ContFut [OK]
XAU_ContFut [OK]
XAV_ContFut [OK]
XAY_ContFut [OK]
ZB_ContFut [OK]
ZC_ContFut [OK]
ZF

### Delete EXO

In [4]:
# storage.delete_exo('VX_ContFut')

# Single EXO analysis

In [5]:
EXO_NAME = 'VX_ContFut' ###
exo_df, exo_dict = storage.load_series(EXO_NAME)

In [6]:
vix = exo_df['exo'].to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/vix.csv')

In [7]:
pwd

'/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas'

In [8]:
vix = pd.DataFrame(pd.read_csv('vix.csv',date_parser=True))
vix.columns = [['date','equity']]
vix.index = vix['date']
vix.index = pd.to_datetime(vix.index)
del vix['date']

In [9]:
vix.to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/vix.csv')

In [10]:
# vix.tail(20).plot()

In [11]:
vix_w_deltas = exo_df[['exo','delta']].to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/vix_w_deltas.csv')

In [12]:
vix_w_deltas = pd.DataFrame(pd.read_csv('vix_w_deltas.csv',date_parser=True))
vix_w_deltas.columns = [['date','equity', 'delta']]
vix_w_deltas.index = vix_w_deltas['date']
vix_w_deltas.index = pd.to_datetime(vix_w_deltas.index)
del vix_w_deltas['date']

In [15]:
vix_w_deltas.tail()

,equity,delta
date,,
2019-02-27,-54400.0,1.0
2019-02-28,-54400.0,1.0
2019-03-01,-54850.0,1.0
2019-03-04,-55400.0,1.0
2019-03-05,-54900.0,1.0


In [16]:
vix_w_deltas.to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/vix_w_deltas.csv')